In this version(1.4): 

- the previous version has the following problems:
    - Repeat the same sentence too many times
    - this problem was solved by randomly choose the candidates in beam search using probabilities, Like in Lottery

In the upcoming versions:
- Evaluation using perplexity
- Beam search, randomize to not repeat the same sentence
- apply to other languages
    - Russian
    - Arabic
    - French
- apply this network for char-level language modeling.
    
For restoring previous check-point, I was forced to define the whole model in this notebook, rather that importing it from an external .py module.

In [1]:
import numpy as np
from lib.dictionarymd import Dictionary
from lib.textprocessingmd import convert_text
from lib.fileoperationmd import getFilesFromPath,readTxtFromFile

[nltk_data] Downloading package stopwords to /home/zein/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from hyper import HyperParameters
hp=HyperParameters()

In [3]:
# Load articles - form dictionary - load dictionary
filenames=getFilesFromPath(hp.DATAPATH)
dataset=[]
text=""
for fn in filenames:
    ntext=convert_text(readTxtFromFile(hp.DATAPATH,fn))
    text+=ntext
    dataset.append(ntext)

dictionary=Dictionary()
dictionary.make_vocab(text,hp.VOCPATH,hp.VOCFILE)
dictionary.load_vocab(hp.VOCPATH,hp.VOCFILE)

199.txt not opened


# Train

In [4]:
import tensorflow as tf
import numpy as np
from RNN_LSTM_LM import LSTMModel
from helpers import batches_generator
from helpers import Beam
from helpers import beam_search,getBestCandidate

tf.reset_default_graph()

/home/zein/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
model = LSTMModel(vocabulary_size=len(dictionary.word2idx), n_tags=len(dictionary.word2idx), embedding_dim=hp.embedding_dim,
                  n_hidden_rnn=hp.n_hidden_rnn, PAD_index=dictionary.word2idx['<PAD>'])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#model_checkpoint = './model.chkpt'
saver = tf.train.Saver()

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
from IPython.display import clear_output
learning_rate=hp.learning_rate
print('Start training... \n')
results=""
start=0
for epoch in range(start,hp.n_epochs):
    newc=getBestCandidate(sess,model,150,"I",dictionary)
    results="\nEpoch {}: {}".format(epoch,newc)
    # For each epoch evaluate the model on train and validation data
    #print('-' * 20 + ' Epoch {} '.format(epoch+1) + 'of {} '.format(hp.n_epochs) + '-' * 20+results)
    
    counter=0
    # Train the model
    for x_batch, y_batch, lengths in batches_generator(hp.batch_size, dataset,dictionary):
        
        clear_output(wait=True)
        counter=counter+1
        print("ُEboch {}/{}.batch {}/{} {}".format(epoch+1,hp.n_epochs,counter,len(dataset)//hp.batch_size,results))
        model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate, hp.dropout_keep_probability)
    # Decaying the learning rate
    saver.save(sess, hp.MODEL_CHKPNT_PATH,global_step=epoch)
    learning_rate = learning_rate / 1.61#hp.learning_rate_decay
    
print('...training finished.')

ُEboch 50/50.batch 511/511 
Epoch 49: <START> I ' m happy to be a good chance to the first set . " #endl , " I ' m not to be in a great opportunity for me . I ' ve been very happy with a great game . I ' ll play in a great game and we are not a good game , I am happy with the way that I was a great player and we have to have a bit nervous . " I am very disappointed and it ' ll have to have a bit more . I am very good for the end , " said the game and we have a great chance . I ' ve had been very happy for a great player . #endl " We ' ve been a good chance . " #endl " We ' re a good performance in the first half . "
...training finished.


In [7]:
%time
# Generate text of 1000 words
full="""The pair are facing lengthy bans for the missed tests, including one on the eve of last year's Athens Olympics. 
They were set to learn their fate by the end of February, but late evidence from them has pushed the date back. "A decision is now expected by around mid-March," said one of their lawyers, Michalis Dimitrakopoulos."""
print(getBestCandidate(sess,model,1000,'I',dictionary))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.2 µs
<START> I have a chance for a good team , and the <UNK> is not going . But he was a great player . " I think it is a lot of the ball , I am not to get a good game . I think we can ' ll play the ball , but I think I was very happy with it , but we have a good team . I think I think it is not to have a lot to be in the game . " I think I ' ve had been a great game . It is the same way to the same - class game . " #endl #endl " The world record of the game , I am happy to play in the end , but I ' ve had been in a lot to play , but we ' ve been very good enough for the game . #endl #endl " We are going out for the club . #endl I have a great game , but it is the best time . It is the most fearsome first two years of their game . " The world record holder , the world record holder is the most fearsome first - half replacement for a <UNK> of their own side , which begins . The 31 . " We ' re looking to be able for me , " she to

In [32]:
#todo

# version2
# output
# normalize probabilities
# Evaluation

# char-level

# larger network

# Russian
# Arabic 
# French

# paper RECURRENT NEURAL NETWORK REGULARIZATION,2015